 # Data Cleaning — E-Commerce Customer Behavior

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Visualization settings
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)


In [5]:
#set visulization style 

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12,6)
print("libraries loaded ")

libraries loaded 


# Data Inspection 

In [ ]:
# Checking the missing values 

missing_summary = 